***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [13]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-4784  GCE       3                                             RUNNING  us-central1-a


# Imports & Setup

In [14]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [15]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Feb 29 17:20 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [17]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [18]:
spark

In [19]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'irprojectaon'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith('.parquet'):
        paths.append(full_path+b.name)

In [20]:
parquetFile = spark.read.parquet(*paths)
# doc_text_pairs = parquetFile.select("text", "id").rdd

In [21]:
# # Count number of wiki pages
# parquetFile.count()

In [22]:
titleDict = parquetFile.select("id", "title").rdd
titleD = titleDict.collectAsMap()
with open('TitleDict.pkl', 'wb') as handle:
   pickle.dump(titleD, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
index_src_title = "TitleDict.pkl"
index_dst_title = f'gs://{bucket_name}/{index_src_title}'
!gsutil cp $index_src_title $index_dst_title

Copying file://TitleDict.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1 files][168.9 MiB/168.9 MiB]                                                
Operation completed over 1 objects/168.9 MiB.                                    


In [24]:
print(len(titleD.keys()))

6348910
